In [ ]:
## Imports 
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import dct,idct,dctn,idctn
import cv2
import sys
import numpy
import PIL
from PIL import Image
numpy.set_printoptions(threshold=sys.maxsize)

## Constants
m = 1
blocksize = 8


In [ ]:

def enCompress(matrix,m = blocksize): 
    """
    enCompress function applys 2d Discrete Cosine Transform (DCT) algoritm on a given square matrix
    param1 : 2d ndarray 
    param2 : integer
    returns : Compressed 2d ndarray of shape (m,m) 
    """
    # return dct(dct(matrix.T,norm = "ortho").T,norm = "ortho")[0:m,0:m]
    return dctn(matrix,norm = "ortho")[:m,:m]

In [ ]:
def deCompress(matrix):
    """
    deCompress function applys Inverse 2d Discrete Cosine Transform (DCT) algoritm on a given square matrix
    param1 : 2d ndarray of shape (x,x)
    returns : De-compressed 2d ndarray of shape (x,x)
    """
    # return idct(idct(matrix.T,norm = "ortho").T,norm = "ortho")
    return idctn(matrix,norm = "ortho")

In [ ]:
def bgr2rgb(imageArray):
    """
    rbg2rgb takes imageArray of mode "RBG" and converts it to "RGB"
    param1 : 3d ndarray shape (x,y,z)
    returns : 3d ndarray shape (x,y,z)
    """
    return np.dstack([imageArray[:,:,2],imageArray[:,:,1],imageArray[:,:,0]])

In [ ]:
## Getting Image
image = cv2.imread(r"image1.png")

## Getting Data from Image
red = image[:,:,0]
blue = image[:,:,1]
green = image[:,:,2]

d1,d2,d3 = image.shape  ## Length of Image

verticalSize = d1//blocksize
horizontalSize = d2//blocksize

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


In [ ]:
## Initializing The RBG arraies of the Comprissed Image
red_of_compressed = np.zeros(((verticalSize) * m,(horizontalSize) * m))
blue_of_compressed = np.zeros(((verticalSize) * m,(horizontalSize) * m))
green_of_compressed = np.zeros(((verticalSize) * m,(horizontalSize) * m))

In [ ]:

## Compressing The Image 
for i in range(0,verticalSize):
    for j in range(0,horizontalSize):

        x = i * blocksize
        y = j * blocksize

        matrix = red[x:x+blocksize,y:y+blocksize]
        red_of_compressed[i*m : (i+1) * m , j*m : (j+1) * m] = enCompress(matrix,m)

        matrix = blue[x:x+blocksize,y:y+blocksize]
        blue_of_compressed[i*m : (i+1) * m , j*m : (j+1) * m] = enCompress(matrix,m)

        matrix = green[x:x+blocksize,y:y+blocksize]
        green_of_compressed[i*m : (i+1) * m , j*m : (j+1) * m] = enCompress(matrix,m)


In [ ]:
compressed_img = np.dstack([red_of_compressed,blue_of_compressed,green_of_compressed])
plt.imshow(compressed_img)

In [ ]:
red_decompressed = np.zeros((d1,d2))
blue_decompressed = np.zeros((d1,d2))
green_decompressed = np.zeros((d1,d2))

In [ ]:
## Decompressing the Image
for i in range(0,verticalSize):
    for j in range(0,horizontalSize):
        matrix = np.zeros((blocksize,blocksize))

        x = i * blocksize
        y = j * blocksize
        matrix[0:m,0:m] = red_of_compressed[i*m : (i+1) * m , j*m : (j+1) * m]
        red_decompressed[x:x+blocksize,y:y+blocksize] = deCompress(matrix)

        matrix[0:m,0:m] = blue_of_compressed[i*m : (i+1) * m , j*m : (j+1) * m]
        blue_decompressed[x:x+blocksize,y:y+blocksize] = deCompress(matrix)

        matrix[0:m,0:m] = green_of_compressed[i*m : (i+1) * m , j*m : (j+1) * m]
        green_decompressed[x:x+blocksize,y:y+blocksize] = deCompress(matrix)


In [ ]:
decompressed_img = image.copy()
decompressed_img[:,:,0] = red_decompressed
decompressed_img[:,:,1] = blue_decompressed
decompressed_img[:,:,2] = green_decompressed
plt.imshow(cv2.cvtColor(decompressed_img, cv2.COLOR_BGR2RGB))

In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))